<a href="https://colab.research.google.com/github/hocine-drioueche/OpenclassroomsProject/blob/main/Light_04_python_rnn_findespace_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://datascientest.fr/train/assets/logo_datascientest.png" style="height:150px">

<hr style="border-width:2px;border-color:#75DFC1">
<h1 style = "text-align:center" > Réseau de neurones récurrents </h1>
<h2 style = "text-align:center" > Trouver les espaces dans une phrase </h2>
<hr style="border-width:2px;border-color:#75DFC1">

Ce notebook est destiné à pratiquer les notions évoquées dans le quatrième exercice du module sur une machine plus adaptée.

Si c'est la première fois que vous utilisez colab, n'hésitez pas à jeter un coup d'oeil sur ce [notebook](https://colab.research.google.com/drive/1jXEKOk3mRYBqFWoVwJ0ZpsRJCWj46Yxt?usp=sharing).


> Dans cet exercice, vous allez vous exercer à utiliser un modèle RNN pour trouver les espaces d'une phrase. En d'autres termes, vous allez trouver l'emplacement des espaces dans une phrase où on les a préalable enlevé:
>
><b>Input</b> : <i>"Iwasawillingworkerandstoodwellwiththeguards."</i>
>
><b>Output</b> : <i>"I was a willing worker and stood well with the guards."</i>


## Modélisation du problème

> Dans cet exercice, nous allons résoudre notre problème en prédisant pour chaque caractère s'il est précédé ou non par un espace.
## Jeu de données


In [ ]:
## Importation des données et packages
!pip install swifter
!python3 -m nltk.downloader stopwords
!python3 -m nltk.downloader punkt
!wget https://train-exo.s3.eu-west-1.amazonaws.com/677/MovieReview.csv
!wget https://train-exo.s3.eu-west-1.amazonaws.com/678/findspace.h5



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 821 kB 36.8 MB/s 
     |████████████████████████████████| 281 kB 71.9 MB/s 
     |████████████████████████████████| 141 kB 67.9 MB/s 
  Created wheel for swifter: filename=swifter-1.3.3-py3-none-any.whl size=16253 sha256=1e0f46dfdb5208abd50f26388074e266cc4d0e032eef08dbfaf6d10b33faab14
  Stored in directory: /root/.cache/pip/wheels/64/bf/da/da0022edab5fd84114858a95e4f32f2fca0d5b7d758905f594
Successfully built swifter
  Attempting uninstall: psutil
    Found existing installation: psutil 5.4.8
    Uninstalling psutil-5.4.8:
      Successfully uninstalled psutil-5.4.8


/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
--2022-08-04 13:18:45--  https://train-exo.s3.eu-west-1.amazonaws.com/677/MovieReview.csv
Resolving train-exo.s3.eu-west-1.amazonaws.com (train-exo.s3.eu-west-1.amazonaws.com)... 52.218.37.128
Connecting to train-exo.s3.eu-west-1.amazonaws.com (train-exo.s3.eu-west-1.amazonaws.com)|52.218.37.128|:443... connec

Un des interêts principaux de colab est la mise à disposition d'un GPU. Utiliser un GPU permet d'accelerer grandement l'execution et donc l'entrainement de modèle de deep learning. Pour configurer le GPU (processeur graphique), il suffit de cliquer sur Edit > Notebook settings et sélectionner GPU comme accélérateur matériel.

* Exécuter la cellule suivante pour vérifier que le GPU soit bien activé.

In [ ]:
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
    print("Please change your hardware accelerator")

Default GPU Device:/device:GPU:0



* Importer le module **pandas** sous le nom **pd**


* Charger le fichier ***'MovieReview.csv'*** sous le nom **df**.


* Afficher les 5 premières lignes du dataframe.

In [ ]:
## Insérez votre code ici

In [ ]:
#@title Solution
import pandas as pd
df = pd.read_csv('MovieReview.csv')
df.head()

,sentiment,review
0,Positive,With all this stuff going down at the moment w...
1,Positive,'The Classic War of the Worlds' by Timothy Hin...
2,Negative,The film starts with a manager (Nicholas Bell)...
3,Negative,It must be assumed that those who praised this...
4,Positive,Superbly trashy and wondrously unpretentious 8...


## Preprocessing

* Appliquer la fonction de preprocessing `preprocess_sentence` à **df.review** pour nettoyer les reviews.

In [ ]:
import re, unicodedata

# Converts the unicode file to ascii
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')

def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())
    w = re.sub(r"[^a-zA-Z?.!,]+", " ", w)
    w = re.sub(r'\b\w{0,1}\b', '', w)
    w = re.sub(r'\ +', " ", w)
    return w

df.review = df.review.apply(lambda x :preprocess_sentence(x))
df.head()

,sentiment,review
0,Positive,with all this stuff going down at the moment w...
1,Positive,the classic war of the worlds by timothy hine...
2,Negative,the film starts with manager nicholas bell giv...
3,Negative,it must be assumed that those who praised this...
4,Positive,superbly trashy and wondrously unpretentious e...


* Stocker dans une variable **text** l'ensemble du texte sous forme d'une chaîne de caractère.

In [ ]:
## Insérez votre code ici



In [ ]:
#@title Solution
from tqdm import tqdm
text = ''
for review in tqdm(df.review):
    text += ' ' + review

100%|██████████| 25000/25000 [00:00<00:00, 290234.33it/s]


* Exécuter la cellule suivante pour définir les variables du vocabulaire.

In [ ]:
import numpy as np
vocab = sorted(list(set(text)))
vocab[0]=''
print('{} uniques characters'.format(len(vocab)))
print(vocab)
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

31 uniques characters
['', '!', ',', '.', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


* Extraire l'ensemble des phrases dans la variable **`sentences`** à l'aide de la fonction `sent_tokenize` de **`nltk.tokenize`**. La fonction `sent_tokenize` permet de découper un texte en une liste de phrases.


* Afficher la distribution du nombre de lettre de chaque phrase.

In [ ]:
## Insérez votre code ici



In [ ]:
#@title Solution
from nltk.tokenize import sent_tokenize
import seaborn as sns
sentences = np.array(sent_tokenize(text))

* Pour éviter des séquences trop long ou trop court, stocker dans la variable **sentences** les phrases entre 50 et 200 caractères.

In [ ]:
## Insérez votre code ici



In [ ]:
#@title Solution
sentences = np.array(sentences)
seuil = 50
seuil2 = 200
sentences = sentences[np.array(list(map(len, sentences)))>seuil]
sentences = sentences[np.array(list(map(len, sentences)))<seuil2]
len(sentences)

207178

* Définir une fonction `conv2input` enlevant tous les espaces d'une liste de phrase.


* Tester la fonction sur la liste de phrase suivante :
[<i>'stories like this with this depth and feeling and this intricacy of meaning are very rare.'</i>,
<i>'it is sad story but i ve never encountered any catharsis more beautifully made.'</i>]

In [ ]:
## Insérez votre code ici



In [ ]:
#@title Solution
def conv2input(sentences):
    targets = []
    for s in sentences:
        targets.append(re.sub(r" ", "", s))
        # Or
        # targets.append(s.replace(" ", ''))
    return targets

conv2input(['stories like this with this depth and feeling and this intricacy of meaning are very rare.',
'it is sad story but i ve never encountered any catharsis more beautifully made.'])

* Convertir l'ensemble des phrases **sentences** dans une variable **inputs** à l'aide de la fonction `conv2target`.

In [ ]:
## Insérez votre code ici



In [ ]:
#@title Solution
inputs = conv2input(sentences)

* Exécuter la cellule pour convertir toutes phrases de **`inputs`** en une liste d'index de caractère. Le résultat sera stocké dans les variables **`sentences_inputs`**, **`max_length_inputs`**.

In [ ]:
def max_length(sentences):
    return max(map(len, sentences))

def process_data(sentences):
    max_len = max_length(sentences)
    text_as_int = np.zeros([len(sentences), max_len], dtype=int)
    for i, s in enumerate(sentences):
        for j, c in enumerate(s):
            text_as_int[i,j] = char2idx[c]
    return text_as_int, max_len


sentences_inputs, max_length_inputs = process_data(inputs)

>
> Puisque dans la partie précédente, nous avons ajouté des 0 pour toutes les séquences aient la même forme, il est nécessaire de définir une troisième classe qui va faire effet de masque sur nos sorties.

* Exécuter la cellule suivante pour définir la variable **`sentences_targets`**.

In [ ]:
def process_target(sentences, max_len):
    text_as_int = np.zeros([len(sentences), max_len], dtype=int)
    for i, s in enumerate(sentences):
        j = -1
        for p in s.split(' ')[:-1]:
            j += len(p)
            text_as_int[i, j]=1
        j += len(s.split(' ')[-1])
        text_as_int[i,j:]=2
    return text_as_int

sentences_targets = process_target(sentences, max_length_inputs)

> Maintenant que les données sont prétraités et mises en forme, il est nécessaire de les séparer en un ensemble d'entraînement et de validation.

* Créer les dataframes **X_train**, **X_test**, **y_train**, **y_test** à partir de **sentences_inputs** et **sentences_targets** en utilisant la fonction `train_test_split` et en gardant 20% des données pour l'échantillon de test. Par souci de reproductibilité des résultats, utiliser random_state=1234.

In [ ]:
## Insérez votre code ici



In [ ]:
#@title Solution
from sklearn.model_selection import train_test_split
# Creating training and validation sets using an 80-20 split
X_train, X_test, y_train, y_test = train_test_split(sentences_inputs, sentences_targets, test_size=0.2, random_state=1234)

# Show length
print('Shape of sentence X :', X_train.shape)
print('Shape of no space sentence y :', y_train.shape)

## Modélisation

Nous allons implementer un **Bidirectionnal RNN**.

* Définir un modèle `Sequential` sous le nom **`model`**.


* Ajouter une couche `Embedding` en précisant une entrée de taille **`len(vocab)`** et une sortie de taille **`len(vocab)`**.


* Ajouter une couche `Bidirectional` composé de `GRU` avec 256 neurones et retournant une séquence (argument **`return_sequences`** de la fonction `GRU`).


* Ajouter une couche `Dropout` afin limiter le sur-apprentissage.


* Ajouter une dernière couche `Dense` de 3 neurones et une fonction d'activation `softmax`.


* Afficher le résumé du modèle.

In [ ]:
## Insérez votre code ici



In [ ]:
#@title Solution
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, RNN, GRUCell, Conv2D, Embedding, Bidirectional, Dropout

model = Sequential()
model.add(Embedding(len(vocab), len(vocab)))
model.add(Bidirectional(RNN(GRUCell(256, recurrent_initializer='glorot_uniform'),
                                return_sequences=True)))
model.add(Dropout(0.3))
model.add(Dense(3, activation='softmax'))
model.summary()


> Comme l'apprentissage d'un algorithme de deep learning est lié à sa fonction perte, nous allons appliquer un masque sur toutes les valeurs de fonction de perte qui ont un label de valeur 2 :
>
> <img src='https://assets-datascientest.s3-eu-west-1.amazonaws.com/notebooks/python_deepnlp_findspace_mask.png' style='width:800px'>
>
> Comme nous sommes face à un problème de classification, nous allons **appliquer notre masque à la fonction de perte sparseCategoricalCrossentropy**.

* Exécuter la cellule suivante pour définir la fonction de perte `loss_function`.

In [ ]:
import tensorflow as tf

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

def loss_function(real, pred):
    # Mask
    mask = tf.math.logical_not(tf.math.equal(real, 2))
    # Avoid type error
    mask = tf.cast(mask, dtype=pred.dtype)
    # Loss function
    loss_ = loss_object(real, pred)
    # Apply mask on loss function
    loss_ *= mask
    return tf.reduce_mean(loss_)

* Compiler le modèle avec la fonction de perte `loss_function` et un optimizer **'adam'**.

In [ ]:
## Insérez votre code ici



In [ ]:
#@title Solution
model.compile(loss=loss_function, optimizer='adam')

* Pour une raison de temps de calcul, nous avons déjà entraîner le modèle. Charger les poids **'findspace.h2'** à l'aide de la méthode `load_weights`.

In [ ]:
## Insérez votre code ici



In [ ]:
#@title Solution
#model.fit(X_train, y_train, batch_size=32, epochs=1)
model.load_weights('findspace.h5')

* Prédire la probabilité de chaque élément de la séquence **X_test[:100]** à l'aide de la méthonde `predict`. Nous prenons que les 100 premiers éléments de **X_test** pour une raison de temps de calcul. Stocker le résultat dans un tableau nommé **y_prob**.


* Appliquer la méthode `argmax` du tableau **y_prob** pour obtenir les classes les plus probable. Il faudra passer l'argument 'axis = 2' pour que l'argmax soit calculée sur chaque élément des séquences. Stocker le résultat dans la variable **y_pred**.

In [ ]:
## Insérez votre code ici
y_prob = model.predict (X_test[:100] )
y_prob = y_prob.argmax (axis=2)


In [ ]:
#@title Solution
y_prob = model.predict(X_test[:100])
y_pred = y_prob.argmax(axis=2)

* Exécuter la cellule suivante pour afficher le résulat du modèle sur les 100 premières phrase de X_test.

In [ ]:
def target2text(X_int, y_pred):
    X = []
    for i in range(len(y_pred)):
        text=''
        for j in np.flip(np.arange(len(y_pred[0]))):
            if y_pred[i, j]==1 :
                text = ' ' + text
            text = idx2char[X_int[i, j]] + text
        X.append(text)
    return X

print('Prediction :')
target2text(X_test[:len(y_pred)], y_pred)

* Exécuter la cellule suivante pour afficher le résultat du modèle sur la phrase <i>"thedatascienceismyway."</i>.

In [ ]:
def process_data(sentences, max_len=None):
    if max_len==None:
        max_len = max_length(sentences)
    text_as_int = np.zeros([len(sentences), max_len], dtype=int)
    for i, s in enumerate(sentences):
        for j, c in enumerate(s):
            text_as_int[i,j] = char2idx[c]
    return text_as_int, max_len

sentences_inputs, _ = process_data(['thedatascienceismyway!'], max_length_inputs)
y_pred = model.predict(sentences_inputs)
target2text(sentences_inputs, y_pred.argmax(axis=2))

## Conclusion

> Cet exercice a essentiellement pour but de s'exercer à l'implémentation un modèle récurant. En effet, trouver les espaces entre les mots n'a pas d'application réelle.
>
> Toutefois, nous pouvons entrevoir les possibilités et la puissance du deep learning sur des tâches liées au texte.